In [3]:
import pandas as pd
import datetime as dt
import requests as req
import csv

In [ ]:
data = pd.read_csv('../data/processed/processed1000.csv', dtype={'Start time': str})
data.head()

,Start date,Start station number,End station number,Bike model,Total duration (m),Start time,start_coordinates,end_coordinates
0,20250131,1142,988,CLASSIC,36,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
1,20250131,1142,988,CLASSIC,36,2359,"51.5034938,-0.07962099","51.51772703,-0.127854211"
2,20250131,300061,300024,CLASSIC,20,2359,"51.495598,-0.078893","51.53658514,-0.075885686"
3,20250131,300058,300076,CLASSIC,4,2359,"51.48512191,-0.174971902","51.4795738,-0.17903854"
4,20250201,300038,300037,CLASSIC,12,0000,"51.48724429,-0.205279052","51.49422354,-0.236769936"


In [6]:
start_journey = data['start_coordinates']
end_journey = data['end_coordinates']
date_journey = data['Start date']
time_journey = data['Start time']

def get_tfl_data(origin, destination, date=None, time=None, timeIs='departing', journeyPreference='leasttime', mode='all'):
    
    url = f'https://api.tfl.gov.uk/Journey/JourneyResults/{origin}/to/{destination}'

    if mode =='all':
        mode = 'bus,overground,tube,coach,dlr,tram,elizabeth-line,replacement-bus'

    params = {
        'date': date,
        'time': time, 
        'timeIs': timeIs, 
        'journeyPreference': journeyPreference,
        'mode': mode
    }

    response = req.get(url, params=params)
    if response.status_code != 200:
        # if response is NOT very feel
        print(f"API returned: {response.status_code}")
        print(f"API returned error {response.status_code} for {origin} to {destination}")
        print(f"Request URL: {response.url}")
        return None
    
    data = response.json()

    if len(data["journeys"]) == 0:
        print(f"No journeys found for {origin} to {destination}")
        return None
    
    # if response is very feel
    return data["journeys"][0]["duration"]


def run_tfl_data():
    duration_values = []

    for i in range(len(data)):
        duration = get_tfl_data(
            origin=start_journey.iloc[i], 
            destination=end_journey.iloc[i], 
            time=time_journey.iloc[i]
        )
        duration_values.append(duration)
    
    trip_duration_dict = {'trip_duration_dict': duration_values}

    return trip_duration_dict

In [10]:
public_commute = run_tfl_data()

data['Public commute'] = public_commute['trip_duration_dict']

In [8]:
# Reorder the DataFrame columns
data_reordered = data[['Start date', 'Start time', 'Start station number', 'End station number', 'Total duration (m)', 'Public commute', 'Bike model', 'start_coordinates', 'end_coordinates']]

data_reordered

,Start date,Start time,Start station number,End station number,Total duration (m),Public commute,Bike model,start_coordinates,end_coordinates
0,20250131,2359,1142,988,36,39,CLASSIC,"51.5034938,-0.07962099","51.51772703,-0.127854211"
1,20250131,2359,1142,988,36,39,CLASSIC,"51.5034938,-0.07962099","51.51772703,-0.127854211"
2,20250131,2359,300061,300024,20,36,CLASSIC,"51.495598,-0.078893","51.53658514,-0.075885686"
3,20250131,2359,300058,300076,4,9,CLASSIC,"51.48512191,-0.174971902","51.4795738,-0.17903854"
4,20250201,0000,300038,300037,12,20,CLASSIC,"51.48724429,-0.205279052","51.49422354,-0.236769936"
5,20250131,2359,1201,200002,21,23,PBSC_EBIKE,"51.51444134,-0.087587447","51.515975,-0.053177"
6,20250131,2358,200012,1031,11,24,CLASSIC,"51.521889,-0.04667","51.507131,-0.06691"
7,20250131,2358,200182,1016,17,29,PBSC_EBIKE,"51.48373225,-0.223852256","51.513735,-0.193487"
8,20250131,2358,2587,2679,8,19,PBSC_EBIKE,"51.51615461,-0.082422399","51.498585,-0.068981764"
9,20250131,2357,300012,300052,2,8,CLASSIC,"51.4737636,-0.197010096","51.475089,-0.201968"


In [9]:
data.to_csv('../data/processed/api_processed/sample_10.csv', quoting=csv.QUOTE_NONNUMERIC)